In [2]:
!pip install praw

     |████████████████████████████████| 188 kB 5.4 MB/s eta 0:00:01


In [30]:
from praw import reddit
import json
import time
import ipfshttpclient
from os import listdir
from os.path import isfile, join
import re
import praw

import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

In [328]:
reddit = praw.Reddit(
    client_id="LtjnfEAH_rRdFDbMwILwxw",
    client_secret="04WSBZHCzqm0ElLYMRkmTkmijCCSog",
    password="PosterThreading69",
    user_agent="my user agent",
    username="postthreadbot",
)

In [3]:
subreddits = "announcements+Art+AskReddit+askscience+atheism+aww+blog+books+creepy+dataisbeautiful+DIY+Documentaries+EarthPorn+explainlikeimfive+food+funny+Futurology+gadgets+gaming+GetMotivated+gifs+history+IAmA+InternetIsBeautiful+Jokes+LifeProTips+listentothis+mildlyinteresting+movies+Music+news+nosleep+nottheonion+OldSchoolCool+personalfinance+philosophy+photoshopbattles+pics+science+Showerthoughts+space+sports+television+tifu+todayilearned+TwoXChromosomes+UpliftingNews+videos+worldnews+WritingPrompts"

In [4]:
substrate = SubstrateInterface(
    url="ws://127.0.0.1:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [61]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair, wait_for_inclusion=True, wait_for_finalization=False):
    call = substrate.compose_call(
        call_module=call_module,  
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=wait_for_inclusion, wait_for_finalization=wait_for_finalization)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except SubstrateRequestException as e:
        print("Failed to send: {}".format(e))
    return receipt

In [243]:
def addSchema(schema, check=True, create=True):
    schemaId = None
    if check:
        schema_count = substrate.query(
            module='Schemas',
            storage_function='SchemaCount',
            params=[]
        ).value

        for i in range(1, schema_count+1):
            schemaTemp = substrate.query(
                module='Schemas',
                storage_function='Schemas',
                params=[i]
            )
            if schemaTemp == schema:
                schemaId = i
                return schemaId

    if schemaId is None and create:
        receipt = make_call("Schemas", "register_schema", {"schema": schema}, bob)
        for event in receipt.triggered_events:
            event = event.decode()
            if event['event']['module_id'] == 'SchemaRegistered':
                schemaId = event['event']['attributes'][1]
                return schemaId
       
        # didn't find msa in events so call function to check for msa
        while (schemaId is None):
            schemaId = addSchema(schema, check=True, create=False)
        return schemaId

In [212]:
def get_msa_id(wallet, create=True, wait_for_inclusion=True, wait_for_finalization=False):
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[wallet.ss58_address]
    )
    
    if not create:
        return None

    if msa_key == None:
        make_call("Msa", "create", {}, wallet, wait_for_inclusion=wait_for_inclusion, wait_for_finalization=wait_for_finalization)
        msa_key = substrate.query(
            module='Msa',
            storage_function='KeyInfoOf',
            params=[wallet.ss58_address]
        )

    msa_id = msa_key['msa_id'].decode()
    return msa_id

In [9]:
def get_signature(payload, signer):
    # encode payload using SCALE
    # I found scale_info from "substrate.metadata_decoder"
    payload_encoded = substrate.encode_scale(type_string='scale_info::8', value=payload['authorized_msa_id']) + \
                            substrate.encode_scale(type_string='scale_info::2', value=payload['permission'])

    # Payload must be wrapped in theses Bytes objects
    payload_encoded = "<Bytes>".encode() + payload_encoded.data + "</Bytes>".encode()

    # The provider address signs the payload, so in this case alice
    return signer.sign(payload_encoded)

In [204]:
def create_msa_with_delegator(provider_wallet, delegator_wallet):    
    provider_msa_id = get_msa_id(provider_wallet)

    payload_raw = { "authorized_msa_id": provider_msa_id, "permission": 0 }

    signature = get_signature(payload_raw, delegator_wallet)

    call_params = {
        "delegator_key": delegator_wallet.ss58_address,
        "proof": {"Sr25519": "0x" + signature.hex()},
        "add_provider_payload": payload_raw
    }

    # provider signs this
    receipt = make_call("Msa", "create_sponsored_account_with_delegation", call_params, provider_wallet)
    for event in receipt.triggered_events:
        event = event.decode()
        if event['event']['module_id'] == 'Msa':
            msa_id = event['event']['attributes'][0]
            return msa_id

In [65]:
def get_data(announcement):
    data = f"{str(announcement.subreddit)},{str(announcement.author.name)},{str(announcement.title)},{str(announcement.selftext)},{str(announcement.url)},{announcement.over_18}"
#     data_hash = substrateinterface.utils.hasher.xxh128(data)
    return data

In [453]:
def mint_votes(user_msa_id, announcement, parent_hash, schemaId):
    receipt_ups = None
    receipt_downs = None
    
    if announcement.ups > 0:
        call_params = {
            "on_behalf_of": user_msa_id,
            "schema_id": schemaId,
            "message": f",{parent_hash},{announcement.ups}"
        }
        receipt_ups = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
    
    if announcement.ups > 0:
        call_params = {
            "on_behalf_of": user_msa_id,
            "schema_id": schemaId,
            "message": f",{parent_hash},-{announcement.downs}"
        }
        receipt_downs = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
    
    return receipt_ups, receipt_downs

In [367]:
def mint_user(user):  
    # create a wallet for user and check if they have an msa id yet
    user_wallet = Keypair.create_from_uri('//' + user.name)
    user_msa_id = get_msa_id(user_wallet, create=False)
    
    # if not then add a user using the schema
    if user_msa_id is None:
        user_msa_id = create_msa_with_delegator(bob, user_wallet)
        user_data = f"{user.name},{user.icon_img}"
        schema = f"username,profile_pic"
        schemaId = addSchema(schema)
        
        call_params = {
            "on_behalf_of": user_msa_id,
            "schema_id": schemaId,
            "message": user_data
        }
        receipt_user = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
    return user_msa_id

In [469]:
def mint_comments(comments, post, post_data_hash, schemaId):
    id_to_hash = {}
    for comment in comments:
        try:
#             comment_user_msa_id = mint_user(comment.author)
            comment_user_msa_id = 1
            username = post.author.name
            profile_pic = post.author.icon_img
        except:
            comment_user_msa_id = 1
            username = "removed"
            profile_pic = "removed"
               
        comment_data = {
            "parent_hash": post_data_hash,
            "username": username,
            "profile_pic": profile_pic,
            "body": comment.body,
        }
        json.dump(comment_data, open(f"./comments/{comment.name}.json", "w"))
        res_comment = client.add(f"./comments/{comment.name}.json")
        comment_data_hash = res_comment['Hash']
        if comment.parent_id == post.name:
            parent_hash = post_data_hash
            comment_schemaId = postcomment_schemaId
        else:
            parent_hash = id_to_hash[comment.parent_id]
            comment_schemaId = commentcomment_schemaId
            
        id_to_hash[comment.id] = comment_data_hash
        message_comment = f",{parent_hash},{comment_data_hash}"
            
        call_params = {
            "on_behalf_of": None,
            "schema_id": comment_schemaId,
            "message": message_comment
        }
        receipt_comment = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
        print(comment_data_hash)
        
        ## comment votes ##
        receipt_ups, receipt_downs = mint_votes(comment_user_msa_id, comment, comment_data_hash, schemaId)
        print(comment.ups, comment.downs)

In [6]:
bob = Keypair.create_from_uri('//Bob')

In [329]:
r_all = reddit.subreddit('all')

In [340]:
r_all.top(time_filter='day')

In [335]:
reddit.front.top()

In [68]:
client = ipfshttpclient.connect()

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [444]:
schema = f"post,category,data_hash"
post_schemaId = addSchema(schema)

schema = f"postcomment,parent_hash,data_hash"
postcomment_schemaId = addSchema(schema)

schema = f"commentcomment,parent_hash,data_hash"
commentcomment_schemaId = addSchema(schema)

schema = f"postvote,parent_hash,votes"
postvote_schemaId = addSchema(schema)

schema = f"commentvote,parent_hash,votes"
commentvote_schemaId = addSchema(schema)

Extrinsic '0xe4fb988a43c1d4d11be87aa6189907d19000248441b4d511a46f35b3f458060d' sent and included in block '0xc03b2e65b359eb5c4639d766e9a486aa38aadd913e6a73baafa448f10e955a11'
Extrinsic '0xd45d3b4b715bdcd0828c1d81424abbf21bf25c62ef40d6f0e0fc4325f83c75e3' sent and included in block '0x1d9f71c3c8b3687db0065109606ae47162bbce4e11c502fa21fa5698f15c459c'
Extrinsic '0xa00fbead41ee1d6badc6afe6ab1812b3abc39308a01be8b361b7abf716f97586' sent and included in block '0x1e779245c4c019697265ea04d69414d1bba9e11e5f88d67b149ca11e64f17f9e'
Extrinsic '0x79ed0d21e8e8efd728e0a26cabba5947069e3c0c393748e79d6752144752ef10' sent and included in block '0xbbf692c0cda5df14bbc2f0cc14515e224043c763e8a0ccdc7cc1f76d366f5755'


In [ ]:
while (True):
    for i, post in enumerate(r_all.top(time_filter='day')):
        ## USER ##
        try:
#             user_msa_id = mint_user(post.author)
            user_msa_id = 1
            username = post.author.name
            profile_pic = post.author.icon_img
        except:
            user_msa_id = 1
            username = "removed"
            profile_pic = "removed"

        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "username": username,
            "profile_pic": profile_pic,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }

        # use hash to check if we already added this post to the blockchain
        # if so then skip
        data_files = [f for f in listdir("./posts") if isfile(join("./posts", f))]
#         if f"{post.name}.json" in data_files:
#             continue

        json.dump(post_data, open(f"./posts/{post.name}.json", "w"))
        res_post = client.add(f"./posts/{post.name}.json")
        post_data_hash = res_post["Hash"]

        # create a schema for all children to use that includes post_hash in order to make easier querying
        # Note: post's will always have parent_hash as null, comments on posts will always have parent_hash as
        # post_data_hash and comments on comments will have their parent comment_data_hash as parent_hash
#         schema = f"{post.subreddit},{post_data_hash},parent_hash,data_hash"
#         schema = f"post,category,parent_hash,data_hash"
#         schemaId = addSchema(schema)

        call_params = {
            "on_behalf_of": None,
            "schema_id": post_schemaId,
            "message": f",{post.subreddit},{post_data_hash}"
        }
        receipt_post = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
        print(post_data_hash)

        ## post votes ##
        receipt_ups, receipt_downs = mint_votes(user_msa_id, post, post_data_hash, postvote_schemaId)
        print(post.ups, post.downs)

        ## COMMENT ##
        mint_comments(post.comments.list()[:10], post, post_data_hash, schemaId)
    #     head_comments = []
    #     for comment in post.comments.list():
    #         if comment.parent_id == post_data_hash:
    #             head_comments[comment].append(comment)
        
    cwsvd
    print("waiting...")
    print()
    # sleep for a bit and then check again
    time.sleep(60*10)

Extrinsic '0xb3be08f8d57662931ec3cb9f462b756835aba8d14534bf1d0bcc5ac24c9f872d' sent and included in block 'None'
QmW86AsSz9gKfHBpgTZikgfBaFH1gYC8r1GPKm2PDrottF
Extrinsic '0x24cc4ebe16dc13b89282ca25618eb26f8da3dd984965e994b64e375d2a2c80c6' sent and included in block 'None'
Extrinsic '0x5eb2560166833f1e91b2f0f999359ad7a89d2d0158a78dfbc9cc75a151bd4cf8' sent and included in block 'None'
130856 0
Extrinsic '0x64caf740f1ff0360498c136b863a5fd8c9eb7381dbb76f3962b7deb2b5654052' sent and included in block 'None'
QmfVaoojeu37b3jMqah1mahrJkjN6GnfQU744MiQdWe5Cu
Extrinsic '0xff5ba7074931ccba457132e8ec114fbbfd7cb7ebb20846f129dd83d3cfe5973c' sent and included in block 'None'
Extrinsic '0xb88a6ca63db753bf92ce2439b048ffd6685267cc247662f4a7d9f29adfc0c1c9' sent and included in block 'None'
1 0
Extrinsic '0x787029c7403275c4fd65ee9df583fd993cec95d379e8728d5c17463d0989993f' sent and included in block 'None'
QmQDmdnabhz2B9i96FRd5T2oLXuXYo2htKR3D1w9QX2bSN
Extrinsic '0x497e4c39f1d095f97b3aaa5ddd98ebf4bdaafc9a17

Extrinsic '0x2b4f52c4a99fde9b91f26407d34f0c9f5fc1371e984a7bfbfc50803887c33fe2' sent and included in block 'None'
QmWoqmQpunMdgR7GsHCinfDRWoeYUx9hy83KnSMdw4Bynd
Extrinsic '0xb44b3ecb9c7f2cb0418f60d24340f05e8d6015bcab869cbd9a9f9d04934708f2' sent and included in block 'None'
Extrinsic '0x11af672de1edb2705f2784e18328a48ce9e0aac8901806f9f26b037cbf8ecdab' sent and included in block 'None'
95953 0
Extrinsic '0xb3181f64b866eca950cb999715cbbba1c1b4afe719484db4b05c6d2875bdd6ce' sent and included in block 'None'
QmShA5qyUgngqgFQdoQNp9NGuncYSJzKmqxyMsM6W4i7Li
Extrinsic '0xc7d1c4534d2ddee34fd3445b5a7535d5835dd11015b003f5153909c9bdd542a3' sent and included in block 'None'
Extrinsic '0xfae4eec64327d4e27dbad36655c4a00920f7fd8de27e56c58d2077733393622d' sent and included in block 'None'
1 0
Extrinsic '0x9d55192bbf8643283d6fc296b4af6393f47a7bcf8156c53389ae81e213990e96' sent and included in block 'None'
Qmd4HFGvyHJCzyUvWtGkzAX3Z5tjZyVEg1oaBHiMvSKBAf
Extrinsic '0x9b474c7450e7cfdb479b000ea6b5eee09f26116ee1a

Extrinsic '0x1984c9ae79dec2ab6b8159df73c786fd91a654587069813c025533ff07908763' sent and included in block 'None'
QmRXG3JcdTeqPE4steUsb7JoJPrKXwrGMaddjVA2MxF5bW
Extrinsic '0xb5380672f61e4efdebd74d82acda4aae9812ed15eb4e2dbcecc0d134a4d6ba0a' sent and included in block 'None'
Extrinsic '0x2a13b10388dc3b32e4d7ba5a041dc286c74d843352f80d8f9df07a93c521ba0f' sent and included in block 'None'
84414 0
Extrinsic '0xc25b855cd881f92e3fd76da630dd246c4a74395d86645ae33824950c536a531b' sent and included in block 'None'
QmbNTMQ4v5z42Q7qtYeVQBXDkwQUXtGNT4FzjcnupeXpCD
Extrinsic '0x14148fd54bc52dea832f441027a13ac2085caf3f511c7f1022d8cdf759f17ce4' sent and included in block 'None'
Extrinsic '0x6ef26954413a4170ca46b16bed8980c3b06236a772f0dec2fda94e288d411625' sent and included in block 'None'
2821 0
Extrinsic '0x83251435d30f08c8c36cd2d202d0a94170f14a4cf8b2f9282d128276f5d39d37' sent and included in block 'None'
Qmc9vManKZEpEBNLQ7X4EmbbSyqxDFrHR8quDg5MUW9DPM
Extrinsic '0x62e9f93a554d68ed86f7801f55e6d8554542c177

Extrinsic '0xd0ca843bf2b2121921b4109ede1ffc432df802d1c693add6442816c44b14c033' sent and included in block 'None'
QmQTLmQQtXgk8VeeK4cZuNGAuix9oxbrH59K6JJww2FYdU
Extrinsic '0x36df43ba8d3a6449b84c8026930230a42c4f9721f8225e8978d019238d00e2e0' sent and included in block 'None'
Extrinsic '0x8536e20247804c571c473ef63e5826245b1a55ef62e03803f55424bd9c63df18' sent and included in block 'None'
73211 0
Extrinsic '0xace1b0c7bb1a4afbe6d1cb70d9b9f6068b903bbd571d9a0d77b845d354b39e7f' sent and included in block 'None'
QmSp8x6SFuXccVPnKaV4heVqwMAJzFZFuhHtgd78vLA5vn
Extrinsic '0x7e6020f505c6a84f373862819b716317bb6332fd342121c151bb3ed253280d9a' sent and included in block 'None'
Extrinsic '0x0b1fb5734c5a72cfbde1b78fe663194ca7eb5694dae9f399b69444a2a55fd72d' sent and included in block 'None'
1 0
Extrinsic '0x053350771f8d1934977a393c662c3c6707099bf225576983a4b9cff613b975c9' sent and included in block 'None'
Qme4XKTG4QXyWFkhYE1VAbCh9Sz7KEcaxSibYMYynptZt5
Extrinsic '0x90e863274605c5242bfdd04ccef3f1f81a4ee8d384d

Extrinsic '0x12752fd6331fd030fca10f06a193e894394f76750bfdb39e9451ad4767fc6f69' sent and included in block 'None'
QmfBGsDABqRZJ7FnPWodNkXsKEi4kg1oJbfPK83Xq6wQEF
Extrinsic '0xc067d8780382d44808b3c2e1e944136684b5f7531e68b72b848f40d69c7122f9' sent and included in block 'None'
Extrinsic '0x56885b74288570468365846afd714df22137d9080a33792ea70203c13ffcaf32' sent and included in block 'None'
72444 0
Extrinsic '0x596b476e4055d7dd751452e963aaac1b36720ba1e47408a25c85616aab7d45ca' sent and included in block 'None'
Qmetk46HCBsNgzeA8hNitFnUPVtXXwuK2JfzZ8arAVH4aD
Extrinsic '0xee235ab4d19c18b189e016c201d6aca3aabf9f02e5fb100dade96088b52568d6' sent and included in block 'None'
Extrinsic '0x8b97850d62806760b32f11808b14433ecd0242ef8d398dc4d4c3a4d5c6682aaa' sent and included in block 'None'
1 0
Extrinsic '0x4672e893d52c3eda3fb9301fef87708498b94ae4de03f925d2fcfcbb8b2e3633' sent and included in block 'None'
QmW6hiZAmxCQGV1V7gG72b4q3qBrVTsZNgPnALcrfCNcVM
Extrinsic '0x5a938ae608e3c7313f69acfe1a2d082efb0769cc653

...

Extrinsic '0x00582425ac3fc5b478184774f393621e419dc1a7fd0535f32e5abd641f0961c6' sent and included in block 'None'
QmYrMybagcC5oMpG92D2XEaNg6rRiMMrnya8SCwxwVHW7g
Extrinsic '0xfd79a4e11e8db7ded92fc05854c2d8a4f654174e392d335c2c03495bf3b91651' sent and included in block 'None'
Extrinsic '0x5177ca54b7a6432b4014532311833715bc924ab89d7c56b810e63e7ccc6a0dca' sent and included in block 'None'
25612 0
Extrinsic '0x0da4f9171ad5472b8f19d5dc885a3ec95dcc1ee3e018a8870569597528f9e0e3' sent and included in block 'None'
QmfYuYww32c8pUmvp3AXrhk2WcJ92sbTYqabbZSmEWvnJ6
Extrinsic '0xa47d6c93e876bd18b0d727b706b5283bb7a27f64a7583b7c2927a5320adf2e05' sent and included in block 'None'
Extrinsic '0xe995ed947ae9f0b760d14d875c04ed105a01e461e58d3bf5b3800d0e0d9995b6' sent and included in block 'None'
1 0
Extrinsic '0xca31d51ba594f08dfc77de884aabcd951171becbcbc681aafe618c32a5a0b5ab' sent and included in block 'None'
QmTx4MF6gu8cjmtAkSarCKrsgpzonrgiJXiJTLt7tVGKf4
Extrinsic '0xc0100be3f10e483390b57560f50fb4047c00dbf5b80

Extrinsic '0xb6b3b1d050419a1b108497e4c010aa04966e4065d1e08aa736fb87ba8ec4f4f1' sent and included in block 'None'
QmaXi7EWYiK3quNLQx7x4d1qcMzG8hU3yMeyvSATnqnBvn
Extrinsic '0x9b82b665f7d28e4e747c5a71c6802694d8f2024d81d731fda22bc43cc5186d17' sent and included in block 'None'
Extrinsic '0x7238a4f93546732e85f9220d630b17cc4ca111b7a084f8bc0249cbb11b632a8f' sent and included in block 'None'
23254 0
Extrinsic '0xfe7cff76b1ac49c3d5f477755a2f077f0a69bd7b238d6aa269a87008efe06ea9' sent and included in block 'None'
Qma398mE8tkHGF8kxSHPQmC8vB2caEXXRA5167cVuvrEfQ
Extrinsic '0x9ebb81118156416bbce2be838d5d31e6cc0830192b96202a2cdb111d724d64c0' sent and included in block 'None'
Extrinsic '0x876018e4eb5a45643da998d23d8d696527c5180588663b23faafda29202a26b0' sent and included in block 'None'
1189 0
Extrinsic '0xffd85bb824896e3172e261eef85212f481d31085deb3ecafcf2f020561df5210' sent and included in block 'None'
QmfUtsDYe3bSfEUxFue2hFqy52PqB7rsZ2TWBNe4kmMWpL
Extrinsic '0xc9fb1bf67527fca4ec71334cbe78159d7a412d10

Extrinsic '0x6dc5c4b3776f83c57eb12602044f7aa12737427025867dbfbea71db7071cf169' sent and included in block 'None'
QmcTMFShwC8REmthp5G6VehKoUiuBh4Fqa4Nt7Rr8ge657
Extrinsic '0x2803272f346df986946ba6371a56d6cb6a55abc348c65d7791042599e28b4388' sent and included in block 'None'
Extrinsic '0x079f0e8e9abaccab1024c771130a2ffffb3397e002763ad5d86b5c79051e5a5f' sent and included in block 'None'
22835 0
Extrinsic '0xe70063dc19f837ebd69ae1432ed6a44323a9da1f05ddfe667249e5b0b540535f' sent and included in block 'None'
QmTQLx7KJrrkvB4V29xjHoUZTdTZXTnWxXeDM7rfscB3FW
Extrinsic '0xee57868059ec1613839c243774b1e034c6d8a215d595f88ea473a9515a2cbf51' sent and included in block 'None'
Extrinsic '0x9e4c72336d6d76559f88e4f8f17eef318602d5b03f91aa1b3054fcd4d084c335' sent and included in block 'None'
1 0
Extrinsic '0xc21ab9e2f01b2c3a30b70a75ca38c4cc06ec9496c3f2f423ed65ce4e5684233a' sent and included in block 'None'
QmSCkDsbsAcfoREqLMyw3eo7yuBYx7WN8y6P18gzihJimb
Extrinsic '0xae286de1fb2c4ede96ee49fb899bfee8a8266e93349

Extrinsic '0x9ffd1c8141466ff2985a7c861de6b4671c1ea082d5334b1250d2899cb8ab6d7d' sent and included in block 'None'
QmQt5bSmLRGBQRfxdQxkCfwRBb6Pgp7cb9qwzHbtXZB1JL
Extrinsic '0x4afed46afba06a3334c0fd14be52ef2c6032d40377cd3f2f8e400e21b4e9285c' sent and included in block 'None'
Extrinsic '0x11ae1e72413516129e71b403814f96d3a6c22e50070981a07fe21d93d29bcc96' sent and included in block 'None'
22686 0
Extrinsic '0xa30407ca2a4131a925bc8fa4be0e9c0e5f1baab3ba41f19de98307fcc1c5d10f' sent and included in block 'None'
QmZKRPJWub7XQbbhxfYi6x4df9zDejdnBH6hYzErGpsT5H
Extrinsic '0xd7463e79e4016dd6b94be764b2e196c4ae75c00fe0fe3abbb6e16082e0abf46b' sent and included in block 'None'
Extrinsic '0x8f9f257a24fc8141e4735cabec5542cca09df2652be28d5a615c4cbbc905178e' sent and included in block 'None'
250 0
Extrinsic '0x09b1360b94dbc6e051aee723192b91ea76991304d15d12128955985dce1d2aad' sent and included in block 'None'
Qmf4KfeopL98XqFEpGbEwCAr3JPHxHSdZzbgKrYiZz1qMR
Extrinsic '0xf845aec590e0df145cfb474b6e8858d11b1df1b5c

Extrinsic '0x35d23e3293e40b8bb390abe2d047ebdf64e642178eb99e6430e9868874b20cd6' sent and included in block 'None'
QmZCtCKVFZvgPDBu4VPwCzKt8nfPBBBg41SYr3ijAUK8Yk
Extrinsic '0x2cd985c1ff5a9acf62ae8ab03e91ed6d3ae8a51adc8fae731398f3cf59058abf' sent and included in block 'None'
Extrinsic '0xd14f5e3636a5ca78d873800bc5b25a79f303667f5a18d814c9cd450d8a82b346' sent and included in block 'None'
22235 0
Extrinsic '0xcb730f1fcce88fbc96b1575c4d58fef7c5f633c91b04c505567b204a94b4f276' sent and included in block 'None'
QmdS6Khd7ofyYJcN3ASqMjYbjswTdVipYDpcRxZWbEahoc
Extrinsic '0xa95abc4acd09ed4a2b0acb4a9efbd02f8c0ea6194e969f86b283f33fcd0354ab' sent and included in block 'None'
Extrinsic '0x51fe1b39be140edcf38483c012c51299e50ace71c7dd2f5784f6c134043828a6' sent and included in block 'None'
631 0
Extrinsic '0x8df47870f660b7686d6d4bacd35e77e8599294f5e0f844e47d448f545975f305' sent and included in block 'None'
QmZWLHQSMK6eQ1CyUcAmXzSLQLZ1NBGGAbRJSMznTJW35V
Extrinsic '0x14f29a013578a7dea1d4014618071deb06b7c2157

In [470]:
substrate.get_block()['header']['number']

5291

In [460]:
start_block = 5291

In [461]:
content_json = {}
for schemaId in [post_schemaId, postcomment_schemaId, commentcomment_schemaId, postvote_schemaId, commentvote_schemaId]:
    params = [
        schemaId,
        {
            "page_size": 10000,
            "from_block": start_block,
            "to_block": 10000,
            "from_index": 0,
        }
    ]

    content_json[schemaId] = substrate.rpc_request(
        method='messages_getBySchema',
        params=params,
    )['result']['content']

In [468]:
content_json[postcomment_schemaId]

[{'block_number': 5282,
  'data': '0x2c516d654838586e75375865353247334a73535042576a453665504c6b4865776f5871654a68737a674e54566b384e2c516d644a436b6a5257383637657a6b48717565416454586a34357775427254535165315564755867665a6d595172',
  'index': 1,
  'msa_id': 1,
  'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'},
 {'block_number': 5282,
  'data': '0x2c516d654838586e75375865353247334a73535042576a453665504c6b4865776f5871654a68737a674e54566b384e2c516d63796a36796377614c33746a5658696754514e4a4a687074796f4463647634706676714a64315867486d3461',
  'index': 2,
  'msa_id': 1,
  'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'},
 {'block_number': 5282,
  'data': '0x2c516d654838586e75375865353247334a73535042576a453665504c6b4865776f5871654a68737a674e54566b384e2c516d65314d31736575614e4856734c77515878713977787a73464c4641543573466f756a6b34326d4a756a636444',
  'index': 3,
  'msa_id': 1,
  'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'},
 {'block_number': 5282,
  'data': 

In [440]:
posts = {}

schema_items = schema.split(',')
category = schema_items[0]
post_hash = schema_items[1]
posts = {}

for content in post_content:
    data = bytes.fromhex(content['data'][2:]).decode().split(',')
    posts[data[2]] = json.loads(client.cat(data[2]).decode())
    
    
for content in vote_content:
    data = bytes.fromhex(content['data'][2:]).decode().split(',')
    if data[1] in posts:
        num_votes = int(data[2])
        if num_votes > 0:
            posts[data[1]]['upvotes'] += num_votes
        else:
            posts[data[1]]['downvotes'] += -1*num_votes   

In [443]:
posts

{'QmW86AsSz9gKfHBpgTZikgfBaFH1gYC8r1GPKm2PDrottF': {'data': {'category': 'SandersForPresident',
   'username': 'Smartercow',
   'profile_pic': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_2.png',
   'title': 'Sanders message to Fox News viewers',
   'body': '',
   'url': 'https://v.redd.it/mzq6430t9n591',
   'is_nsfw': False},
  'upvotes': 130282,
  'downvotes': 0},
 'QmYrMybagcC5oMpG92D2XEaNg6rRiMMrnya8SCwxwVHW7g': {'data': {'category': 'HolUp',
   'username': 'BonBonBonne',
   'profile_pic': 'https://styles.redditmedia.com/t5_3jjgud/styles/profileIcon_snoo7e777884-fa7d-4a49-9c7b-1253d4e27009-headshot.png?width=256&height=256&crop=256:256,smart&s=173a28d46cf0b506807b9c12194b3ea4bf560ab5',
   'title': 'Dang, it took a while for this lady to realise this',
   'body': '',
   'url': 'https://i.redd.it/viv8fmkybr591.png',
   'is_nsfw': False}},
 'QmfLFZgJ5ZHKheBT74wDWPPiPTXWmw7PcELTgu2Z77UEf4': {'data': {'category': 'HumansBeingBros',
   'username': 'Sir_Loinbeef',
   '

In [395]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

post_schemas = {}
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    print(schemaTemp.value)

username,profile_pic
pics,QmVbFm5mNfgk1o4e6tuhRDrygWhynnabq9aQfcZSW42RPY,parent_hash,data_hash
wallstreetbets,QmYwaTsoKb6jF8y5AGresVKj5LzypWZPWGwyzyeSgVYjy4,parent_hash,data_hash
news,QmZRYUyrppGsDPDB2Q9wqyfJAtPkWKbTkSs9FasxGLpvi7,parent_hash,data_hash
movies,QmdfkFFtwxGyZaLw7teURyuxGmzpPxLjZrUYSYUfT874VL,parent_hash,data_hash
funny,Qmf943KpByMd3Bz6j2QG4CppVHgG2SV93u7wzHwyiMGVVL,parent_hash,data_hash
wallstreetbets,QmdZZ1RcE7z8eiqS4e687qm9gbsGZBsAGYsbpBr1xMsMyv,parent_hash,data_hash
memes,QmUm4yRmjbgyjeeKukuGPkYiZJRB1KeR9J1u7vWLus9j2w,parent_hash,data_hash
funny,QmTLrPfhVysDXEsUZdqz7GYZ6xF1Xy7FL7SUDeb2SxaejQ,parent_hash,data_hash
aww,QmUJxZmYJhZDoJFS1vcEv86kXtjN1rSym9Ywh7QtfBLzZC,parent_hash,data_hash
wallstreetbets,QmYpWJBnpnyZmvQpp4XyHtbtnKLP9VG9hwUchUmrXTwMq6,parent_hash,data_hash
pics,Qmd8ZYMiWohnEpEW2SQjTsFJZaj1HfnBeUENBkM4hAqASV,parent_hash,data_hash
memes,QmRcBBbh54LN1tVzjX4aKC6skNPgC8rj1sup3SkLWZSLwr,parent_hash,data_hash
gaming,QmZsjUdDvXffGETG6oVAwtJFRs7jmnG4vaFRbpP1sRn9HD,pa

In [349]:
post_pattern = re.compile(f".*,.*,parent_hash,data_hash")

In [350]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

post_schemas = {}
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if post_pattern.match(schemaTemp.value):
        print(schemaTemp.value)
        post_schemas[schemaTemp.value] = i

pics,QmVbFm5mNfgk1o4e6tuhRDrygWhynnabq9aQfcZSW42RPY,parent_hash,data_hash
wallstreetbets,QmYwaTsoKb6jF8y5AGresVKj5LzypWZPWGwyzyeSgVYjy4,parent_hash,data_hash
news,QmZRYUyrppGsDPDB2Q9wqyfJAtPkWKbTkSs9FasxGLpvi7,parent_hash,data_hash
movies,QmdfkFFtwxGyZaLw7teURyuxGmzpPxLjZrUYSYUfT874VL,parent_hash,data_hash
funny,Qmf943KpByMd3Bz6j2QG4CppVHgG2SV93u7wzHwyiMGVVL,parent_hash,data_hash
wallstreetbets,QmdZZ1RcE7z8eiqS4e687qm9gbsGZBsAGYsbpBr1xMsMyv,parent_hash,data_hash
memes,QmUm4yRmjbgyjeeKukuGPkYiZJRB1KeR9J1u7vWLus9j2w,parent_hash,data_hash
funny,QmTLrPfhVysDXEsUZdqz7GYZ6xF1Xy7FL7SUDeb2SxaejQ,parent_hash,data_hash
aww,QmUJxZmYJhZDoJFS1vcEv86kXtjN1rSym9Ywh7QtfBLzZC,parent_hash,data_hash
wallstreetbets,QmYpWJBnpnyZmvQpp4XyHtbtnKLP9VG9hwUchUmrXTwMq6,parent_hash,data_hash
pics,Qmd8ZYMiWohnEpEW2SQjTsFJZaj1HfnBeUENBkM4hAqASV,parent_hash,data_hash
memes,QmRcBBbh54LN1tVzjX4aKC6skNPgC8rj1sup3SkLWZSLwr,parent_hash,data_hash
gaming,QmZsjUdDvXffGETG6oVAwtJFRs7jmnG4vaFRbpP1sRn9HD,parent_hash,data_hash
g

In [363]:
substrate.get_block()['header']['number']

In [364]:
b['header']['number']

5123

In [287]:
b.keys()

dict_keys(['extrinsics', 'header'])

In [361]:
%%time
content_jsons = {}
for schema, schemaId in post_schemas.items():
    params = [
        schemaId,
        {
            "page_size": 10000,
            "from_block": 5123,
            "to_block": 10000,
            "from_index": 0,
        }
    ]

    content = substrate.rpc_request(
        method='messages_getBySchema',
        params=params,
    )
    if len(content['result']['content']) > 0:
        content_jsons[schema] = content['result']['content']

CPU times: user 276 ms, sys: 7.44 ms, total: 283 ms
Wall time: 11.5 s


In [362]:
len(content_jsons)

116

In [353]:
def add_comment(_data, _dict, _is_vote):
    for hsh in _dict.keys():
        if hsh == data[2]:
            if is_vote:
                if '-' in data[3]:
                    # downvote
                    _dict['data']['downvotes'] += data[3]
                else:
                    # upvote
                    _dict['data']['upvotes'] += data[3]
            else:
                _dict[data[3]] = {
                    'data': json.loads(client.cat(_data).decode()), 
                    'comments': {}
                }
                
                _dict['data'].update({"upvotes": 0, "downvotes": 0})
            return True
        result = add_comment(_data, _dict['comments'], _is_vote)
        if result:
            return True
    return False

In [354]:
posts = {}
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    category = schema_items[0]
    post_hash = schema_items[1]
    posts[post_hash] = {'data': {"upvotes": 0, "downvotes": 0}, 'comments': {}}
    
    for content in contents:
        data = bytes.fromhex(content['data'][2:]).decode().split(',')
        # votes had numeric numbers instead of hashes
        is_vote = data[3].strip('-').isnumeric() 
        if is_vote:
            data[3] = int(data[3])
            
        if data[2] == '': 
            # empty means its post data
            posts[post_hash]['data'].update(json.loads(client.cat(data[3]).decode()))
        elif data[2] == post_hash: 
            # post_hash means its comment
            if is_vote:
                if data[3] < 0:
                    # downvote
                    posts[post_hash]['data']['downvotes'] += data[3]
                else:
                    # upvote
                    posts[post_hash]['data']['upvotes'] += data[3]
            else:
                posts[post_hash]['comments'][data[3]] = {
                    'data': json.loads(client.cat(data[3]).decode()), 
                    'comments': {}
                }
                posts[post_hash]['comments'][data[3]]['data'].update({"upvotes": 0, "downvotes": 0})
        else: 
            # neither means its comment of comment
            result = add_comment(data[3], posts[post_hash]['comments'], is_vote)
            if not result:
                print('couldnt find', data[2], json.loads(client.cat(data[3]).decode()))

In [355]:
from collections import OrderedDict
o = OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True))

In [356]:
o.items()

odict_items([('QmT2Zo15TiDYGqfmXAmYkMGkf4WMY8sdQGiNsZsHAtuYnN', {'data': {'upvotes': 1167863, 'downvotes': 0, 'category': 'wallstreetbets', 'username': 'SomeGuyInDeutschland', 'profile_pic': 'https://styles.redditmedia.com/t5_bcx7b/styles/profileIcon_snoo908b3c30-edde-42cf-b40d-83391a012cb0-headshot-f.png?width=256&height=256&crop=256:256,smart&s=98c80967daec2a3af329ef021215bf076e0456e3', 'title': 'Times Square right now', 'body': '', 'url': 'https://v.redd.it/x64z70f7eie61', 'is_nsfw': False}, 'comments': {'QmTcQ8sFZPCn4Hp9TAgq2Uwi8Cbo6UZAGktyMvN3U4dnxL': {'data': {'parent_hash': 'QmT2Zo15TiDYGqfmXAmYkMGkf4WMY8sdQGiNsZsHAtuYnN', 'username': 'SomeGuyInDeutschland', 'profile_pic': 'https://styles.redditmedia.com/t5_bcx7b/styles/profileIcon_snoo908b3c30-edde-42cf-b40d-83391a012cb0-headshot-f.png?width=256&height=256&crop=256:256,smart&s=98c80967daec2a3af329ef021215bf076e0456e3', 'body': 'Someone needs to open a GameStop across the ny stock exchange 💎🙌', 'upvotes': 0, 'downvotes': 0}, 'co

In [357]:
posts_list = []
for k, v in OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True)).items():
    posts_list.append({k:v})

In [358]:
posts_list

[{'QmT2Zo15TiDYGqfmXAmYkMGkf4WMY8sdQGiNsZsHAtuYnN': {'data': {'upvotes': 1167863,
    'downvotes': 0,
    'category': 'wallstreetbets',
    'username': 'SomeGuyInDeutschland',
    'profile_pic': 'https://styles.redditmedia.com/t5_bcx7b/styles/profileIcon_snoo908b3c30-edde-42cf-b40d-83391a012cb0-headshot-f.png?width=256&height=256&crop=256:256,smart&s=98c80967daec2a3af329ef021215bf076e0456e3',
    'title': 'Times Square right now',
    'body': '',
    'url': 'https://v.redd.it/x64z70f7eie61',
    'is_nsfw': False},
   'comments': {'QmTcQ8sFZPCn4Hp9TAgq2Uwi8Cbo6UZAGktyMvN3U4dnxL': {'data': {'parent_hash': 'QmT2Zo15TiDYGqfmXAmYkMGkf4WMY8sdQGiNsZsHAtuYnN',
      'username': 'SomeGuyInDeutschland',
      'profile_pic': 'https://styles.redditmedia.com/t5_bcx7b/styles/profileIcon_snoo908b3c30-edde-42cf-b40d-83391a012cb0-headshot-f.png?width=256&height=256&crop=256:256,smart&s=98c80967daec2a3af329ef021215bf076e0456e3',
      'body': 'Someone needs to open a GameStop across the ny stock exchang

In [305]:
dict(sorted(posts.items()))

{'QmPFmSNa1kmQLR4tYwJG5DuJC4HiBNnfu3vHw8tUCjR8xr': {'data': {'upvotes': 70712,
   'downvotes': 0},
  'comments': {'Qmb7ukkEQ3jPXgVnVC5F1JNQmWvCJxC2jYYhHzansj6c5Z': {'data': {'parent_hash': 'QmPFmSNa1kmQLR4tYwJG5DuJC4HiBNnfu3vHw8tUCjR8xr',
     'username': 'ReallyRickAstley',
     'profile_pic': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_4.png',
     'body': 'For a second I thought this was /r/OldSchoolCool and was thinking ‘Wow, he looks just like Rick Astley’',
     'upvotes': 0,
     'downvotes': 0},
    'comments': {}},
   'QmacZuUZnTEvJEiYT2AF7TiS7v4Y2EicYWS5TyH6W9d5FU': {'data': {'parent_hash': 'QmPFmSNa1kmQLR4tYwJG5DuJC4HiBNnfu3vHw8tUCjR8xr',
     'username': 'ReallyRickAstley',
     'profile_pic': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_4.png',
     'body': "A reminder that Rick gave the best AMA answer ever a few years back. [No it's not a rickroll.](https://www.reddit.com/r/Music/comments/56cdgm/ama_im_really_rick_astley_i_swear_a

In [272]:
data

['', '', 'QmQeMR1Zvx14bHU1M19a85MDrhQCVuU1Y8jWZLYi8v5kbH', 0]

In [260]:
s = '-352'

In [264]:
s.strip('-').isnumeric()

True